In [2]:
import requests
import time
import firebase_admin
from firebase_admin import credentials, firestore
from tqdm import tqdm

import grpc
from google.cloud.firestore_v1.client import firestore_client
from google.cloud.firestore_v1.client import firestore_grpc_transport

In [3]:
# Determine the environment and load the appropriate .env file
env = "development"
if env == "development":
  cred = credentials.Certificate('dev_firebase_config.json')
  firebase_app = firebase_admin.initialize_app(cred)

  channel = grpc.insecure_channel("localhost:8080")
  transport = firestore_grpc_transport.FirestoreGrpcTransport(channel=channel)
  db = firestore.client(app=firebase_app)
  db._firestore_api_internal = firestore_client.FirestoreClient(transport=transport)

else:
  firebase_app = firebase_admin.initialize_app()
  db = firestore.client(app=firebase_app)

In [4]:
# Initialize Firebase Admin using a service account
def fetch_all_courses(base_url, initial_page=1, sleep_time=1):
    courses = []
    page = initial_page

    initial_response = requests.get(f"{base_url}?page={page}")
    if initial_response.status_code != 200:
        print(f"Failed to fetch initial data: HTTP {initial_response.status_code}")
        return courses

    # total_pages = initial_response.json()['universita']['totalPages']
    total_pages = 5
    
    for page in tqdm(range(initial_page, total_pages + 1), desc="Fetching pages"):
        url = f"{base_url}?page={page}"
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to fetch data on page {page}: HTTP {response.status_code}")
            break

        data = response.json()
        courses.extend(data['universita']['corsi'])
        time.sleep(sleep_time)

    return courses

def save_to_firestore(courses):
    collection_ref = db.collection(u'courses')
    for course in courses:
        # Use document IDs from the course data or let Firestore generate them
        doc_ref = collection_ref.document()
        doc_ref.set(course)
    print("Data has been written to Firestore.")

# Example usage
base_url = "https://universitaly-backend.cineca.it/api/offerta-formativa/cerca-corsi"
all_courses = fetch_all_courses(base_url)

Fetching pages: 100%|██████████| 5/5 [00:11<00:00,  2.23s/it]


In [5]:
# Save to Firebase
save_to_firestore(all_courses)

PermissionDenied: 403 
false for 'create' @ L6, false for 'update' @ L6